In [ ]:
import os
from typing import Dict, List
from dotenv import load_dotenv
from openai import OpenAI
from IPython.display import display, Markdown, clear_output

class ChatInterface:
    def __init__(self):
        self._initialize_environment()
        self._setup_openai()
        self.system_prompt = "You are a smart LLM which can answer complex questions in a concise and clear way"
        
    def _initialize_environment(self) -> None:
        """Initialize environment variables"""
        load_dotenv(override=True)
        self.api_key = os.getenv('OPENAI_API_KEY')
        if not self._validate_api_key():
            raise ValueError("Invalid API key. Please check your .env file")

    def _validate_api_key(self) -> bool:
        """Validate the API key format"""
        return (self.api_key 
                and self.api_key.startswith('sk-proj-') 
                and len(self.api_key) > 10)

    def _setup_openai(self) -> None:
        """Setup OpenAI client"""
        self.client = OpenAI()
        self.model = "gpt-4o-mini"

    def get_user_input(self) -> str:
        """Get formatted user input"""
        return input("Ask me anything: ").strip() + "\n\n"

    def format_messages(self, user_input: str) -> List[Dict[str, str]]:
        """Format messages for the API call"""
        return [
            {"role": "system", "content": self.system_prompt},
            {"role": "user", "content": user_input}
        ]

    def stream_response(self, messages: List[Dict[str, str]]) -> None:
        """Stream and display the model's response"""
        stream = self.client.chat.completions.create(
            model=self.model,
            messages=messages,
            stream=True
        )
        
        # Initialize display
        response = ""
        display_handle = display(Markdown(""), display_id=True)
        
        # Stream response
        for chunk in stream:
            if chunk.choices[0].delta.content:
                response += chunk.choices[0].delta.content
                # Clean up markdown artifacts
                cleaned_response = self._clean_markdown(response)
                # Update display with formatted response
                self._update_display(cleaned_response, display_handle)

    def _clean_markdown(self, text: str) -> str:
        """Clean up markdown formatting artifacts"""
        return text.replace("```", "").replace("markdown", "")

    def _update_display(self, text: str, handle) -> None:
        """Update the display with formatted text"""
        handle.update(Markdown(text))

    def run(self) -> None:
        """Main execution loop"""
        try:
            while True:
                user_input = self.get_user_input()
                if user_input.lower().strip() in ['quit', 'exit']:
                    break
                messages = self.format_messages(user_input)
                self.stream_response(messages)
                print("\n" + "-"*50 + "\n")  # Visual separator between interactions
                
        except KeyboardInterrupt:
            print("\nGoodbye!")

if __name__ == "__main__":
    chat = ChatInterface()
    chat.run()